In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers
from sklearn.metrics import accuracy_score, mean_squared_error
from math import sqrt
from keras.utils import to_categorical
import pandas as pd
from PIL import Image
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.metrics import mean_squared_error, r2_score
from keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error
import numpy as np

In [2]:
import os

train_folder = "H:/BIG DATA/nn_final_project/dataset/preprocessed-boneage-training-dataset"
test_folder = "H:/BIG DATA/nn_final_project/dataset/boneage-test-dataset"

# Count the number of folders in the train and test directories
num_classes_train = len(os.listdir(train_folder))
num_classes_test = len(os.listdir(test_folder))

print("Number of classes in training data:", num_classes_train)
print("Number of classes in testing data:", num_classes_test)

# Choose the maximum number of classes between train and test data
num_classes = max(num_classes_train, num_classes_test)
print("Total number of classes:", num_classes)

Number of classes in training data: 2647
Number of classes in testing data: 1
Total number of classes: 2647


In [3]:
from keras.utils import to_categorical
import pandas as pd
from PIL import Image

# Load the metadata from the CSV file
csv_file_path = "H:/BIG DATA/nn_final_project/dataset/boneage-training-dataset.csv"
df = pd.read_csv(csv_file_path)

# Path to the folder containing the training images
train_folder = "H:/BIG DATA/nn_final_project/dataset/preprocessed-boneage-training-dataset"

# Lists to store the image data and bone age labels
x_train = []
y_train = []

# Loop through the rows in the DataFrame
for index, row in df.iterrows():
    image_id = row['id']
    image_filename = f"{image_id}.png"  # Assuming the image filename is stored as an integer 'id'
    bone_age = row['boneage']
    image_path = os.path.join(train_folder, image_filename)
    if os.path.isfile(image_path):
        try:
            # Open and preprocess the image
            img = Image.open(image_path)
            # Preprocess the image as needed (e.g., resize, normalization)
            img = np.array(img)
            img = img.astype("float32") / 255.0
            # Add the preprocessed image and bone age label to the lists
            x_train.append(img)
            y_train.append(bone_age)
        except Exception as e:
            print(f"Error processing image: {image_filename}, Error: {e}")

# Convert the lists to numpy arrays
x_train = np.array(x_train)
y_train = np.array(y_train)

print(x_train)
print(y_train)

[[[0.4862745  0.4862745  0.4862745  ... 0.9019608  0.9843137  0.99607843]
  [0.4862745  0.4862745  0.4862745  ... 0.9019608  0.9843137  0.99607843]
  [0.4862745  0.4862745  0.4862745  ... 0.9019608  0.9843137  0.99607843]
  ...
  [0.73333335 0.73333335 0.73333335 ... 0.73333335 0.73333335 0.73333335]
  [0.73333335 0.73333335 0.73333335 ... 0.8352941  0.8392157  0.8392157 ]
  [0.73333335 0.73333335 0.73333335 ... 0.99215686 0.99215686 0.99215686]]

 [[0.21960784 0.21960784 0.21960784 ... 0.         0.         0.        ]
  [0.21960784 0.21960784 0.21960784 ... 0.         0.         0.        ]
  [0.21960784 0.21960784 0.21960784 ... 0.         0.         0.        ]
  ...
  [0.4        0.4        0.4        ... 0.         0.         0.        ]
  [0.4        0.4        0.4        ... 0.         0.         0.        ]
  [0.4        0.4        0.4        ... 0.         0.         0.        ]]

 [[0.8039216  0.8039216  0.79607844 ... 0.59607846 0.59607846 0.60784316]
  [0.73333335 0.733333

In [4]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.metrics import mean_squared_error, r2_score
from keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error
import numpy as np

# Normalize the images to [0, 1]
x_train = x_train.astype("float32") / 255.0

# Reshape the data to match the CNN input shape (assuming grayscale images)
input_shape = x_train[0].shape + (1,)
x_train = x_train.reshape(-1, *input_shape)

# Split the data into training and validation sets
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=42)

# Build your CNN model architecture
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='linear'))  # Output layer for regression (bone age)

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model with the validation data
history = model.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_val, y_val))

# Predict bone age labels for the training data
y_train_pred = model.predict(x_train)

# Calculate SSR (Sum of Squared Residuals) on training data
ssr_train = np.sum((y_train_pred - y_train)**2)

# Calculate MSE (Mean Squared Error) on training data
mse_train = mean_squared_error(y_train, y_train_pred)

# Calculate RMSE (Root Mean Squared Error) on training data
rmse_train = np.sqrt(mse_train)

# Calculate R-squared on training data
r2_train = r2_score(y_train, y_train_pred)

# Evaluate the model on the validation data
y_val_pred = model.predict(x_val)

# Calculate SSR (Sum of Squared Residuals) on validation data
ssr_val = np.sum((y_val_pred - y_val)**2)

# Calculate MSE (Mean Squared Error) on validation data
mse_val = mean_squared_error(y_val, y_val_pred)

# Calculate RMSE (Root Mean Squared Error) on validation data
rmse_val = np.sqrt(mse_val)

# Calculate R-squared on validation data
r2_val = r2_score(y_val, y_val_pred)

# Print validation metrics
print("Validation SSR:", ssr_val)
print("Validation MSE:", mse_val)
print("Validation RMSE:", rmse_val)

# Print training metrics
print("Training SSR:", ssr_train)
print("Training MSE:", mse_train)
print("Training RMSE:", rmse_train)

h:\BIG DATA\nn_final_project\venv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 74s 882ms/step - loss: 7550.3496 - val_loss: 1804.5425
Epoch 2/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 64s 855ms/step - loss: 1472.9541 - val_loss: 1783.1729
Epoch 3/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 78s 1s/step - loss: 1566.6573 - val_loss: 1799.9706
Epoch 4/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 88s 1s/step - loss: 1569.0623 - val_loss: 1941.1333
Epoch 5/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 129s 937ms/step - loss: 1535.4979 - val_loss: 1856.0170
Epoch 6/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 93s 1s/step - loss: 1478.4456 - val_loss: 1766.1206
Epoch 7/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 122s 971ms/step - loss: 1597.7880 - val_loss: 1754.5330
Epoch 8/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 91s 1s/step - loss: 1602.3843 - val_loss: 1752.8374
Epoch 9/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 90s 1s/step - loss: 1551.7172 - val_loss: 1786.0359
Epoch 10/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 81s 1s/step - loss: 1531.6990 - val_loss: 1749.1874
75/75 ━━━━━━━━━━━━━━━━━━━━ 20s 272ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 137ms/step
Vali

In [6]:
import pickle

In [8]:

# Example of saving the model to a file using pickle
model_file = "model.pkl"
file_path = 'H:/BIG DATA/nn_final_project'
# Define a function to save the model
def save_model_to_pickle(model, file_path):
    with open(file_path, 'wb') as f:
        pickle.dump(model, f)

# Save the model
save_model_to_pickle(model, model_file)